# Tensorflow Basic Flow
### Excerpted from DeepLearning Online Course
Excerpted from Morvan
https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/2-2-example2/

### 1. Basic Framework

In [1]:
import tensorflow as tf
import numpy as np

#create data
x_data = np.random.randn(100,10).astype(np.float32)
y_data = np.random.randn(5,10)

In [3]:
#Step 1: Create variable type

#1. constant (numbers)
a = tf.constant(5, name='const1')

#2. variable (train parameters)
weights1 = tf.Variable(tf.random_uniform([5,100],-1.0,1.0), dtype=tf.float32, name='w1') #input real value first, allow same name
bias1 = tf.Variable(tf.zeros([5,1]))

with tf.variable_scope("scope1", reuse=False):
    weights2 = tf.get_variable('w1', [1], initializer=tf.contrib.layers.xavier_initializer(seed=1)) #input name first, don't allow same name
    bias2 = tf.get_variable('b1', [1], initializer=tf.zeros_initializer())

#3. placeholder (data)
x = tf.placeholder(tf.float32, [100,1], name='x1')

In [4]:
#step 2: Calculation

"""
    funciton list:
    1. math
    tf.add, tf.subtract, tf.matmul, tf.square
    2. shape
    tf.get_shape().as_list(), tf.reshape(X, [dimension1, dimension2, -1]), tf.transpose(X, perm=[0,1,2]) //height, row, column
    3. sum
    tf.reduce_sum, tf.reduce_mean
    4. function
    tf.nn.relu, tf.nn.softmax, tf.sigmoid
"""

z = tf.matmul(weights1,x_data)+bias1
a = tf.tanh(z)
cost = tf.reduce_mean(tf.square(a-y_data))

In [5]:
#step 3: Setup initializer

init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.04)
train = optimizer.minimize(cost)

In [6]:
#step 4: Create session
sess = tf.Session()

sess.run(init)

for i in range(1000):
    sess.run([train,cost])
    
    if i % 10 == 0:
        print(i, sess.run(cost))

sess.close()

#or use with tf.Session() as sess:

0 1.427361
10 1.2352552
20 1.1726582
30 1.1141702
40 1.1081862
50 1.1052725
60 1.1029644
70 1.1004261
80 1.0964683
90 1.08881
100 1.0716075
110 1.0093932
120 0.9972276
130 0.9644335
140 0.9532713
150 0.9351319
160 0.9289573
170 0.92738074
180 0.9266496
190 0.92630935
200 0.92604774
210 0.92578614
220 0.9254988
230 0.92516065
240 0.9247297
250 0.92412513
260 0.9231668
270 0.92139846
280 0.91769964
290 0.91222864
300 0.9101879
310 0.909927
320 0.90985656
330 0.90980715
340 0.9097622
350 0.90971947
360 0.9096784
370 0.90963876
380 0.9096004
390 0.9095631
400 0.90952677
410 0.9094908
420 0.9094555
430 0.9094206
440 0.90938574
450 0.90935105
460 0.90931594
470 0.9092806
480 0.90924454
490 0.90920746
500 0.9091693
510 0.90912956
520 0.90908784
530 0.90904355
540 0.9089962
550 0.9089447
560 0.9088884
570 0.9088257
580 0.9087547
590 0.9086732
600 0.90857744
610 0.90846246
620 0.9083203
630 0.9081389
640 0.90789694
650 0.90755713
660 0.9070449
670 0.906193
680 0.90456843
690 0.90092444
700 0.89

### 2. Basic Neural Network

In [22]:
def add_layer(inputs, in_size, out_size, activation=None):
    
    Weights = tf.Variable(tf.random_normal([out_size, in_size]))
    Bias = tf.Variable(tf.zeros([out_size, 1]))
    z = tf.matmul(Weights,inputs)+Bias
    
    if activation == None:
        a = z
    else:
        a = activation(z)
    
    return a

In [8]:
x_data = np.random.randn(4,10)
noise = np.random.normal(0, 0.05, x_data.shape).astype(np.float32)
y_data = np.square(x_data)+noise

In [17]:
x = tf.placeholder(tf.float32, [4,10], name='x')
y = tf.placeholder(tf.float32,  [None, 10], name='y')

l1 = add_layer(x, 4, 10, activation=tf.nn.relu)
l2 = add_layer(l1, 10, 4, activation=tf.nn.softmax)

loss = tf.reduce_mean(tf.reduce_sum(tf.square(l2-y_data), reduction_indices=[0]))

(4, 10)


In [10]:
init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)
train = optimizer.minimize(loss)

In [11]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(train, feed_dict={x: x_data, y: y_data})
        if i % 50 is 0:
            print(sess.run(loss, feed_dict={x: x_data, y: y_data}))

15.82314
12.81499
12.811424
12.810336
12.809816
12.809515
12.809321
12.809186
12.809084
12.809008
12.808948
12.808897
12.808859
12.808825
12.808797
12.808772
12.808751
12.808733
12.808718
12.808701


### 3. MNIST DATASET

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [45]:
# There are 55000 pictures, with dimension 28x28
# 1. 1 Dense layer
Set X,Y
X = tf.placeholder(tf.float32, [784, None], name='X')
Y = tf.placeholder(tf.float32, [10, None], name='Y')
a = add_layer(X, 784, 10, activation=tf.nn.softmax)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(a), reduction_indices=[1]))

init = tf.global_variables_initializer()
train = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

sess = tf.Session()
sess.run(init)
losses = []

for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    batch_xs = (batch_xs).T
    batch_ys = (batch_ys).T
    sess.run(train, feed_dict={X: batch_xs, Y: batch_ys})
    if i % 50 is 0:
        loss = sess.run(cross_entropy, feed_dict={X: batch_xs, Y: batch_ys})
        losses.append(loss)

print(losses)
sess.close()

[152.93726, 107.56612, 82.62453, 71.28184, 65.80028, 74.89519, 70.787346, 61.97777, 60.352303, 68.14247, 56.086487, 53.849438, 56.302906, 52.936657, 52.55849, 48.54546, 58.553284, 49.337, 51.537495, 45.76406, 47.85145, 48.035656, 45.560253, 48.759327, 43.58686, 41.005817, 47.064034, 45.24936, 42.78026, 49.010956, 41.55939, 47.268555, 39.18636, 43.344486, 42.182564, 45.108932, 44.705406, 43.35429, 42.92578, 37.307487]


In [82]:
# 2. CNN layer
# Define variables
X = tf.placeholder(tf.float32, [None, 784], name='X')
Y = tf.placeholder(tf.float32, [None, 10], name='Y')
x_image = tf.reshape(X, [-1,28,28,1])
keep_prob = tf.placeholder(tf.float32)

with tf.variable_scope("input", reuse=False):
    w1 = tf.get_variable('w1', [5, 5, 1, 32], initializer = tf.contrib.layers.xavier_initializer(seed=1))
    b1 = tf.get_variable('b1', [32], initializer = tf.zeros_initializer())
    w2 = tf.get_variable('w2', [5, 5, 32, 64], initializer = tf.contrib.layers.xavier_initializer(seed=1))
    b2 = tf.get_variable('b2', [64], initializer = tf.zeros_initializer())
    w3 = tf.get_variable('w3', [7*7*64,1024], initializer = tf.contrib.layers.xavier_initializer(seed=1))
    b3 = tf.get_variable('b3', [1024], initializer = tf.zeros_initializer())
    w4 = tf.get_variable('w4', [1024, 10], initializer = tf.contrib.layers.xavier_initializer(seed=1))
    b4 = tf.get_variable('b4', [10], initializer = tf.zeros_initializer())


In [101]:
# Define layers
z1 = tf.nn.conv2d(x_image, w1, strides=[1,1,1,1], padding='SAME')+b1
a1 = tf.nn.relu(z1)
p1 = tf.nn.max_pool(a1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

z2  = tf.nn.conv2d(p1, w2, strides=[1,1,1,1], padding='SAME')+b2
a2 = tf.nn.relu(z2)
p2 = tf.nn.max_pool(a2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

p2_flat = tf.reshape(p2, [-1,7*7*64])
z3 = tf.matmul(p2_flat, w3)+b3
a3 = tf.nn.relu(z3)
a3_drop = tf.nn.dropout(a3, keep_prob)

pred = tf.nn.softmax(tf.matmul(a3_drop, w4)+b4)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(pred)+(1-Y)*tf.log(1-pred),reduction_indices=[1]))

In [106]:
# Define tensor
train = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
init = tf.global_variables_initializer()

% matplotlib inline
import matplotlib.pyplot as plt

In [104]:
# Create session
sess = tf.Session()
sess.run(init)

for i in range(500):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.6})
    if i % 50 is 0:
        loss = sess.run(cross_entropy, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.6})
        print(loss)

sess.close()

3.1986659
1.3343158
1.012357
0.45507276
0.39604196
0.42146033
0.33032155
0.18361716
0.47104943
0.1786221


In [4]:
#3. RNN layer
#Define variables
batch_size = 128
n_x = 28
T_x = 28
n_a = 128
n_y = 10

X = tf.placeholder(tf.float32, [None, T_x, n_x], name='X')
Y = tf.placeholder(tf.float32, [None, n_y], name='Y')
weights = {'in': tf.Variable(tf.random_normal([n_x, n_a])), 'out': tf.Variable(tf.random_normal([n_a, n_y]))}
biases = {'in': tf.constant(0.1, shape=[n_a, ]), 'out': tf.Variable(0.1, [n_y, ])}

In [8]:
#Define layers
def RNN(X, weights, biases):
    
    X=tf.reshape(X, [-1, n_x])
    X_in = tf.matmul(X, weights['in'])+biases['in']
    X_in = tf.reshape(X_in, [-1, T_x, n_x])
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_a, forget_bias=1.0, state_is_tuple=True, reuse=None)
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)
    results = tf.matmul(final_state[1], weights['out'])+biases['out']
    
    return results

In [9]:
#Define tensor
pred = RNN(X, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
train = tf.train.AdamOptimizer(1e-3).minimize(cost)
init = tf.global_variables_initializer()
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
#Create session
with tf.Session() as sess:
    sess.run(init)
    step = 0
    
    while step*batch_size < 10000:
        
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = np.reshape(batch_xs,[batch_size, T_x, n_x])
        sess.run([train], feed_dict={X: batch_xs, Y: batch_ys})
        
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys}))
        
        step += 1

InvalidArgumentError: Input to reshape is a tensor with 458752 values, but the requested shape requires a multiple of 784
	 [[Node: Reshape_3 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](add_1, Reshape_3/shape)]]

Caused by op 'Reshape_3', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "/ana